In [2]:
from mujoco_py import load_model_from_path, MjSim
import numpy as np
import os
import re
from shared_utils.general import gen_models_folder_path

#### Load model

In [3]:
model_path = os.path.join(
        gen_models_folder_path(), "exo_with_patient", "nesm_with_simple_patient.xml")
assert os.path.isfile(model_path)

In [4]:
model = load_model_from_path(model_path)
sim = MjSim(model)
sim.forward()

#### Active contacts

Length of `sim.data.active_contacts_efc_pos` varies in time; however no clear idea on what it is exactly

In [5]:
sim.data.active_contacts_efc_pos.shape

(6017,)

#### MjContact object

MjContact objects are stored in `sim.data.contact` ; its size is equal to `nconmax`, the maximal number of contacts

In [9]:
assert len(sim.data.contact) == model.nconmax

I guess that unactive contacts are those were `contact_efc_address` == 0 or equivalently those were geom1 / geom2 == 0

In [11]:
# active contacts
active_contacts = [con for con in sim.data.contact if con.efc_address != 0]
print(len(active_contacts))

1375


Strangely it seems that the number of active contacts doesn't change.

In [24]:
# check if it varies in time
for _ in range(100):
    sim.step()
    
active_contacts = [con for con in sim.data.contact if con.efc_address != 0]
print(len(active_contacts))


contact_geoms = np.array([[con.geom1, con.geom2] for con in active_contacts])

1375


In [25]:
contact_geoms

array([[ 10,  11],
       [ 10,  15],
       [ 10,  16],
       ...,
       [367, 370],
       [368, 369],
       [369, 370]])

### Show contact between upper arm support and upper arm soft collider

In [26]:
ua_supp_reg = re.compile("^ua_[0-9]+$")
ua_soft_reg = re.compile("^uaG[0-9]+_[0-9]+_[0-9]+$")

ua_supp_gnames = [name for name in model.geom_names if ua_supp_reg.match(name)]
ua_soft_gnames = [name for name in model.geom_names if ua_soft_reg.match(name)]

ua_supp_gidx = np.vectorize(model.geom_name2id)(ua_supp_gnames)
ua_soft_gidx = np.vectorize(model.geom_name2id)(ua_soft_gnames)

In [27]:
assert np.max(ua_soft_gidx) < np.min(ua_supp_gidx)
print(f"Support geom indexes are strictly superior to that of the soft collider")

Support geom indexes are strictly superior to that of the soft collider


In [28]:
# get addresses of contacts involving the upper arm support and the upper arm collider
contact_geoms = np.sort(contact_geoms, axis=1)
# print(contact_geoms)

ua_contacts = np.array([con for con in active_contacts if con.geom2 in ua_supp_gidx and con.geom1 in ua_soft_gidx])
ua_contacts.shape

(36,)

In [34]:
print(ua_contacts[0].dist)
print(ua_contacts[0].pos)
print(ua_contacts[0].dim)
print(ua_contacts[0].efc_address)

-0.0035230366061999264
[-0.02960466 -0.02964512 -0.08930004]
3
997


In [35]:
sim.data.efc_force[ua_contacts[0].efc_address]

3.9364729356589327